In [1]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

True

## Init models

In [2]:
sentiment_analysis_model = ChatGroq(model="llama-3.3-70b-versatile", model_kwargs={"response_format": {"type": "json_object"}})
feature_extraction_model = ChatGroq(model="gemma2-9b-it",  model_kwargs={"response_format": {"type": "json_object"}})

## Create prompt templates

In [3]:
sentiment_analysis_system_message = SystemMessage("You are an emotion analysis bot. Your job is to perform sentiment analysis on the given text and identify the top 5 emotions present. " \
                                                    "Your response should be in the form of a JSON object with a single attribute called 'sentiments' " \
                                                    "which contains an array of the top 5 emotions in order from most prevalent to least prevalent.")
sentiment_analysis_template = ChatPromptTemplate.from_messages([sentiment_analysis_system_message, ("user", "{input}")])

In [4]:
feature_extraction_system_message = SystemMessage("You are a textual analysis bot. Your job is to find every phrase in the given text that may indicate the intent for action. " \
                                                    "Return a JSON object containing every phrase that you find to be actionable. Be liberal with your analysis, as these can be phrases without verbs.")
feature_extraction_template = ChatPromptTemplate.from_messages([feature_extraction_system_message, ("user", "{input}")])

## Invoke models

In [5]:
sentiment_prompt = sentiment_analysis_template.invoke({"input": "I am so happy today, I feel like I can conquer the world!"})
sentiment_prompt

ChatPromptValue(messages=[SystemMessage(content="You are an emotion analysis bot. Your job is to perform sentiment analysis on the given text and identify the top 5 emotions present. Your response should be in the form of a JSON object with a single attribute called 'sentiments' which contains an array of the top 5 emotions in order from most prevalent to least prevalent.", additional_kwargs={}, response_metadata={}), HumanMessage(content='I am so happy today, I feel like I can conquer the world!', additional_kwargs={}, response_metadata={})])

In [6]:
extraction_prompt = feature_extraction_template.invoke({"input": "I am going to the store to buy some milk. I also need to pick up some bread and eggs. Then at 7:30 I will go to the gym for a workout."})
extraction_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a textual analysis bot. Your job is to find every phrase in the given text that may indicate the intent for action. Return a JSON object containing every phrase that you find to be actionable. Be liberal with your analysis, as these can be phrases without verbs.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I am going to the store to buy some milk. I also need to pick up some bread and eggs. Then at 7:30 I will go to the gym for a workout.', additional_kwargs={}, response_metadata={})])

In [7]:
sentiment_analysis = sentiment_analysis_model.invoke(sentiment_prompt)
print(sentiment_analysis.content)

{
   "sentiments": [
       "Happiness",
       "Excitement",
       "Optimism",
       "Confidence",
       "Euphoria"
   ]
}


In [8]:
feature_extraction = feature_extraction_model.invoke(extraction_prompt)
print(feature_extraction.content)

{
  "actionable_phrases": [
    "going to the store",
    "buy some milk",
    "pick up some bread",
    "pick up some eggs",
    "go to the gym",
    "workout"
  ]
}
